### (2024 dataset) Fetch CSV from URL & Connect AWS database

##### STEP 1: Fetch the CSV files from the URL (available monthly, 12 urls needed for yearly data). 
Note: This URL is for testing and only contains SPP nodes from 2024. In another jupyter notebook, the URL will be engineered to include the latest monethly data for all nodes to streamline the automation process. 

In [ ]:
import requests
import pandas as pd
from io import StringIO

#Define the function that output a dataframe object from a given url
def getDf_fromURL(url):
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the CSV content using pandas
        csv_data = StringIO(response.text)  # Convert response content to a file-like object
        df = pd.read_csv(csv_data)
        return df
    else:
        return print(f"Failed to fetch the data. Status code: {response.status_code}")

In [ ]:
#url with year and month as variables in f-string, obtain data for all 12 months for that year
year='2024'
get_LMPdata = pd.DataFrame()
for month in ('01','02','03','04','05','06','07','08','09','10','11','12'):
    URL=f'https://portal.spp.org/file-browser-api/download/rtbm-lmp-by-location?path=/{year}/{month}/RTBM-LMP-MONTHLY-SL-{year}{month}.csv'
    newDF = getDf_fromURL(URL)    
    get_LMPdata = pd.concat([get_LMPdata, newDF], ignore_index=True)
get_LMPdata

##### STEP 2: Dataframe Manipulation with Python

The dataframe is transfromed into the intended form, with 5 columns: </p>
   
    1. Datetime (date-time data type)
    2. Settlement Location Name (string type)
    3. LMP (float type)
    4. MLC (float type)
    5. MCC (float type)

In [ ]:
#Define the function that restructure the dataframe 
def conversion(raw_data):
    # formate dataframe 
    raw_data.columns = raw_data.columns.str.strip()

    melted = raw_data.melt(
        id_vars=['Date', 'Settlement Location Name', 'Price Type'],
        value_vars=[f'HE{i:02d}' for i in range(1, 25)],
        var_name='hour',
        value_name='price'
    )

    # Convert hour (e.g., HE01) to a time
    melted['hour'] = melted['hour'].str.extract('HE(\d+)').astype(int) - 1
    melted['Datetime'] = pd.to_datetime(melted['Date']) + pd.to_timedelta(melted['hour'], unit='h')

    # Pivot `price_type` to create `lmp`, `mcc`, `mlc` columns
    result = melted.pivot_table(
        index=['Datetime', 'Settlement Location Name'],
        columns='Price Type',
        values='price',
        aggfunc='first'
    ).reset_index()

    return result  

In [ ]:
#Test with the sample LMP data
fin_LMP = conversion(get_LMPdata)

In [ ]:
#correct datetime column dtype
fin_LMP['Datetime'] = pd.to_datetime(fin_LMP['Datetime'])
#filter to year 2024
#fin_LMP = fin_LMP[fin_LMP['Datetime'].dt.year==2024]
#ignore this step - all monthly data is ranging from 6:00 in the first month the 1st to 5:00 on the next month the 1st

In [ ]:
# add month, day, and hour columns for merging later
fin_LMP['month']=fin_LMP['Datetime'].dt.month
fin_LMP['day']=fin_LMP['Datetime'].dt.day
fin_LMP['time']=fin_LMP['Datetime'].dt.hour

In [ ]:
fin_LMP

In [ ]:
#upload dataframe to database
fin_LMP.to_sql('raw_lmp', engine, if_exists='replace', index=False)
print('raw_lmp stored in the database')

##### STEP 3: Adding Coordinate Columns
Inner join coordinates table with LMP dataframe. The final dataframe contains 7 columns: </p>
   
    1. datetime (date-time data type)
    2. Settlement Location Name (string type)
    3. LMP (float type)
    4. MLC (float type)
    5. MCC (float type)
    6. latitude (float type)
    7. longitude (float type)

In [ ]:
#import dataframe of pnodes with coordinate
coords = pd.read_csv('spp_nodes_buses_coords_new.csv')

In [ ]:
len(pd.unique(coords['SETTLEMENTLOCATION']))

In [ ]:
###### add coor table to database ######

#upload dataframe to database
coords.to_sql('node_coords', engine, if_exists='replace', index=False)
print('node_coords stored in the database')

In [ ]:
#inner join the dataframes
LMP_df = pd.merge(fin_LMP, coords, 
                     left_on='Settlement Location Name', 
                     right_on='SETTLEMENTLOCATION',
                     how='inner')
LMP_df = LMP_df.filter(['Datetime', 'Settlement Location Name', 'LMP', 'MLC', 'MCC', 'latitude', 'longitude'],axis=1)
LMP_df

##### STEP 4: Adding Solar Output
Inner join solar production table with LMP_df. The final dataframe contains 8 columns:

    1. datetime (date-time data type)
    2. Settlement Location Name (string type)
    3. LMP (float type)
    4. MLC (float type)
    5. MCC (float type)
    6. latitude (float type)
    7. longitude (float type)
    8. output

In [ ]:
#import solar production table
solar = pd.read_csv('SPP_pNodes_solar_production.csv')
solar

In [ ]:
# count number of nodes
len(pd.unique(solar['node_id']))

In [ ]:
# add month, day, and hour columns for merging later
solar['hour'] = pd.to_datetime(solar['hour'])
solar['month']=solar['hour'].dt.month
solar['day']=solar['hour'].dt.day
solar['time']=solar['hour'].dt.hour

In [ ]:
solar

In [ ]:
###### add solar table to database ######

#store the solar production table in database
solar.to_sql('solar_output', engine, if_exists='replace', index=False)
print('solar_output stored in the database')

In [ ]:
#inner join the dataframes
final_df = pd.merge(LMP_df, solar, 
                     left_on=['Settlement Location Name','month','day','time'], 
                     right_on=['node_id','month','day','time'],
                     how='inner')
final_df = final_df.filter(['Datetime', 'Settlement Location Name', 'LMP', 'MLC', 'MCC', 'latitude', 'longitude','output'],axis=1)
final_df

##### Upload data to database

In [ ]:
#Create the engine with connection string (provided in Supabase) to establish connection with the database

from sqlalchemy import create_engine
connection_string = 'postgresql://postgres:PW@...'
engine = create_engine(connection_string)

# Test the connection
with engine.connect() as connection:
    result = connection.execute("SELECT NOW();")
    for row in result:
        print(row)

In [ ]:
#store the final_df in database
final_df.to_sql('final_df', engine, if_exists='replace', index=False)
print('final_df stored in the database')

In [ ]:
engine.dispose()

##### (side project) testing daytime and nighttime lmp

In [ ]:
daytime = final_df[final_df['output']>0]
nighttime = final_df[final_df['output']==0]

Ave_dLMP = daytime.groupby('Settlement Location Name')['LMP'].mean()
Ave_nLMP = nighttime.groupby('Settlement Location Name')['LMP'].mean()

In [ ]:
Differences_df = pd.DataFrame({
    'Settlement Location Name': Ave_nLMP.index,
    'Ave Daytime LMP': Ave_dLMP.values, 
    'Ave Nighttime LMP': Ave_nLMP.values,
    'LMP Difference (Daytime - Nighttime)': Ave_dLMP.values-Ave_nLMP.values
})
Differences_df